# TODO :  change from sorting by resource type to profile

## Create FHIR R4 SearchParameter Resource

Create FHIR R4 SearchParameter Resource, Quick start text, and Searchparameter list using the python fhir client

Source data is in excel file

### Prerequisites:

- Python 3.7 or greater


### Import FHIRR4Client and other libraries

In [1]:
%config IPCompleter.greedy=True

In [2]:
from fhirclient.r4models.fhirabstractbase import FHIRValidationError
from fhirclient.r4models import searchparameter as SP
from fhirclient.r4models import capabilitystatement as CS
from fhirclient.r4models import bundle as B
import fhirclient.r4models.identifier as I
import fhirclient.r4models.coding as C
import fhirclient.r4models.codeableconcept as CC
import fhirclient.r4models.fhirdate as D
import fhirclient.r4models.extension as X
import fhirclient.r4models.contactdetail as CD
from json import dumps, loads, load
from requests import get, post, put
import os
from pathlib import Path
from csv import reader as csvreader
from IPython.display import display as Display, HTML, Markdown
from pprint import pprint
from collections import namedtuple
from pandas import *
from datetime import datetime
from jinja2 import Environment, FileSystemLoader, select_autoescape
import R4sp_summary_list as sp_map
from stringcase import snakecase, titlecase, pascalcase
from itertools import zip_longest
from openpyxl import load_workbook

####  Assign Global Variables


Here is where we assign all the global variables for this example such as the local paths for file input and output

##### Need to update:
- base_id
- paths
- canonical

In [3]:
#******************** Need to update *************************************************
fhir_base_url = 'http://hl7.org/fhir/'
base_id = "US-Core"
canon_base = "http://hl7.org/fhir/us/core/"
ig_folder = 'US-Core'
publisher = 'HL7 International - Structured Documents Work Group'
publisher_endpoint = dict(
                        system = 'url',
                        value = 'http://www.hl7.org/Special/committees/structure/index.cfm'
                        ) 
#ig_source_path = "//ERICS-AIR-2/ehaas/Documents/FHIR/US-Core-R4/source/"
ig_source_path = "/Users/ehaas/Documents/FHIR/US-Core-R4/source/"
#ig_source_path = ''
ig_pages_path= '/Users/ehaas/Documents/FHIR/US-Core-R4/source/pages/_includes'
skip_types = ['Questionnaire']
#***********************************************************************************

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )

md_template = ['quick_start.j2', 'sp_list_page.j2', 'cs_search_documentation.j2','sp_narrative.j2']


fhir_term_server = 'http://test.fhir.org/r3'

# profile = 'http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient' # The official URL for this profile is: http://hl7.org/fhir/us/core/StructureDefinition/us-core-patient



none_list = ['', ' ', 'none', 'n/a', 'N/A', 'N', 'False', 'FALSE']
sep_list = (',', ';', ' ', ', ', '; ')



### validate

In [4]:
# *********************** validate Resource as Dict ********************************

def validate(r):

    fhir_test_server = 'http://test.fhir.org/r4'

    headers = {
    'Accept':'application/fhir+json',
    'Content-Type':'application/fhir+json'
    }
    
    params = {
    }
    
    r = post(f'{fhir_test_server}/{r["resourceType"]}/$validate', params = params, headers = headers, data = dumps(r))
    # return r.status_code
    # view  output
    # return (r.json()["text"]["div"])
    return r

### Get Search Parameter input data

In [5]:
#in_path = '/Users/ehaas/Documents/FHIR/pyfhir/test/'
in_path =''

in_file ="uscore-server"

xls = ExcelFile(f'{in_path}{in_file}.xlsx')
df = read_excel(xls,'sps',na_filter = False)
df_combos = read_excel(xls,'sp_combos',na_filter = False)

df_combos

,index,base,combo,combo_conf,types,description,example,imp_note
0,1,Encounter,"class,date",SHOULD,"date,token",,,
1,2,!Encounter,"class,date,patient",SHOULD,"date,reference,token",,,
2,3,!Encounter,"class,date,patient,type",SHOULD,"date,reference,token",,,
3,4,!Encounter,"class,date,type",SHOULD,"date,token",,,
4,5,Encounter,"class,patient",SHOULD,"reference,token",,,
5,6,!Encounter,"class,patient,status",SHOULD,"reference,token",,,
6,7,!Encounter,"class,patient,status,type",SHOULD,"reference,token",,,
7,8,!Encounter,"class,patient,type",SHOULD,"reference,token",,,
8,9,!Encounter,"class,status",SHOULD,token,,,
9,10,!Encounter,"class,status,type",SHOULD,token,,,


In [6]:
data = [i for i in df.itertuples(index=True)]
r_type =  {d.base for d in data if '!' not in d.base}
combo_data = [i for i in df_combos.itertuples(index=True)]

for d in data:
    print(f'Resource = {d.base}, Search Parameter = {d.code}, Exists = {d.exists}')
for c in combo_data:
    print(f'Resource = {c.base}, Combo Search Parameter = {c.combo}')

Resource = !Patient, Search Parameter = address, Exists = Y
Resource = !Patient, Search Parameter = telecom, Exists = Y
Resource = AllergyIntolerance, Search Parameter = clinical-status, Exists = Y
Resource = AllergyIntolerance, Search Parameter = patient, Exists = Y
Resource = Condition, Search Parameter = category, Exists = Y
Resource = Condition, Search Parameter = clinical-status, Exists = Y
Resource = Condition, Search Parameter = patient, Exists = Y
Resource = Encounter, Search Parameter = _id, Exists = Y
Resource = Encounter, Search Parameter = class, Exists = Y
Resource = Encounter, Search Parameter = date, Exists = Y
Resource = Encounter, Search Parameter = identifier, Exists = Y
Resource = Encounter, Search Parameter = patient, Exists = Y
Resource = Encounter, Search Parameter = status, Exists = Y
Resource = Encounter, Search Parameter = type, Exists = Y
Resource = Patient, Search Parameter = _id, Exists = Y
Resource = Patient, Search Parameter = birthdate, Exists = Y
Resourc

### update core SP with additional capabiliities


- Get definitions bundle and convert to python object for ease of notation
- use sp_map to map to Type + parameter
- If need to update SP Extract the SP based on the excel file

### load SP Mapping dictionary

output shows a single SP entry for a sanity check

In [7]:
#p = Path('C:/Users/Eric/Documents/HL7/FHIR/BUILD_EDIT_FILES/R4_Definitions/search-parameters.json')
p = Path('/Users/ehaas/Downloads/definitionsR4.json/search-parameters.json')
b = B.Bundle(loads(p.read_text()), strict = False)
sp = (i.resource for i in b.entry if i.resource.url == 'http://hl7.org/fhir/SearchParameter/Resource-id')
sp_for_id = next(sp)
print(dumps(sp_for_id.as_json(), indent = 4))

{
    "id": "Resource-id",
    "extension": [
        {
            "url": "http://hl7.org/fhir/StructureDefinition/structuredefinition-standards-status",
            "valueCode": "trial-use"
        }
    ],
    "base": [
        "Resource"
    ],
    "code": "_id",
    "contact": [
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://hl7.org/fhir"
                }
            ]
        },
        {
            "telecom": [
                {
                    "system": "url",
                    "value": "http://www.hl7.org/Special/committees/fiwg/index.cfm"
                }
            ]
        }
    ],
    "date": "2018-12-27T22:37:54+11:00",
    "description": "Logical id of this artifact",
    "experimental": false,
    "expression": "Resource.id",
    "modifier": [
        "missing",
        "text",
        "not",
        "in",
        "not-in",
        "below",
        "above",
        "ofType"
    ],

## create updated SPs
sp optional 'modifier' elements are listed as comma separated list of shalls and shoulds for each:
- multipleOr
- multipleOr_conf
- multipleAnd
- multipleAnd_conf
- shall_modifier
- should_modifier
- shall_comparator
- should_comparator
- shall_chain

if not spedified then conformance is MAY 

*note that it starts out using the FHIRClient models but the switches to a dict structure to add the FHIR primitive type extensions.*

In [8]:
def sp_expectation(conf=None):
    if not conf:
        conf = "MAY"
 
    x = X.Extension(dict(
    url = f'http://hl7.org/fhir/StructureDefinition/capabilitystatement-expectation',
    valueCode = conf
    ))
    x_dict = dict(
    extension = [x.as_json()]
    ) 
    return x_dict

def find_base_sp(d):
    if '!' not in d.base and d.update =='Y':
        #print(d.base, d.code)
        if d.code == '_id':
            sp = SP.SearchParameter(sp_for_id.as_json())
        else:
            sp = next(i.resource for i in b.entry if i.resource.code == d.code and d.base in i.resource.base)       
            sp = SP.SearchParameter(sp.as_json())
        return sp


sp_list=[]
for d in data:
        print(f'========={d.base},{d.code}==============')
        sp = find_base_sp(d)
        #print(type(sp))
        if sp:
            #print(sp.url)
            # change id and url, publisher, and contact, draft etc
            sp.id = f'{base_id.lower()}-{d.base.lower()}-{d.code.replace("_","")}'  
            sp.extension = []
            sp.derivedFrom =sp.url
            sp.url = f'{canon_base}SearchParameter/{sp.id}'
            sp.publisher = publisher
            sp.contact = [CD.ContactDetail( {"telecom" : [ publisher_endpoint ] })]
            sp.date = D.FHIRDate(f'{datetime.utcnow().isoformat()}Z')
            sp.name = pascalcase(sp.id.replace('-','_'))
            sp.name = sp.name.replace('UsCore','USCore')
            sp.status = 'active'

            sp.base = [d.base]
            
            try:
                sp.expression = [i for i in sp.expression.split('|') if i.startswith(f'{d.base}.')][0]
            except IndexError:
                try:
                    sp.expression = [i for i in sp.expression.split(' | ') if i.startswith(f'{d.base}.')][0]
                except IndexError:
                    sp.expression = f'{d.base}.id'
            try:
                sp.xpath = [i for i in sp.xpath.split('|') if i.startswith(f'f:{d.base}/')][0]
                sp.xpath = sp.xpath.strip()
            except IndexError:
                try:
                    sp.xpath = [i for i in sp.xpath.split(' | ') if i.startswith(f'f:{d.base}/')][0]
                    sp.xpath = sp.xpath.strip()
                except IndexError:
                    sp.xpath = f'{d.base}.id'
            except AttributeError:
                pass
            try:   
                sp.description = [i for i in sp.description.split('\r\n* ') if i.startswith(f'[{d.base}]')][0]
                sp.description = sp.description.split(': ')[1:]
                sp.description = ''.join(sp.description)
            except IndexError:   
                print(sp.expression)
            sp.expression = sp.expression.strip()
            sp.description = sp.description.strip() 

  
            #convert to dict since model can't handle primitive extensions
            sp_dict = sp.as_json()

            sp_dict['multipleOr'] = False if d.multipleOr in none_list else True
            sp_dict['_multipleOr'] = sp_expectation(d.multipleOr_conf)
            
            sp_dict['multipleAnd'] = False if d.multipleAnd in none_list else True
            sp_dict['_multipleAnd'] = sp_expectation(d.multipleAnd_conf)

            try:
                sp_dict['_modifier'] = []
                for m in sp_dict['modifier']: # list all modifiers in sp and assign an expectation.
                    if d.shall_modifier not in none_list and m in d.shall_modifier.split(','):
                       sp_dict['_modifier'].append(sp_expectation('SHALL'))
                    elif  d.should_modifier not in none_list and m in d.should_modifier.split(','):
                        sp_dict['_modifier'].append(sp_expectation('SHOULD'))               
                    else:
                        sp_dict['_modifier'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_modifier'])

            try:
                sp_dict['_comparator'] = []
                for m in sp_dict['comparator']: # list all comparators in sp and assign an expectation.
                   if d.shall_comparator not in none_list and m in d.shall_comparator.split(','):
                       sp_dict['_comparator'].append(sp_expectation('SHALL'))
                   elif  d.should_comparator not in none_list and m in d.should_comparator.split(','):
                        sp_dict['_comparator'].append(sp_expectation('SHOULD'))               
                   else:
                        sp_dict['_comparator'].append(sp_expectation('MAY'))
            except KeyError:
                del(sp_dict['_comparator'])

            if d.shall_chain not in none_list:
               sp_dict['chain'] = d.shall_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.shall_chain.split(',')]

            if d.should_chain not in none_list:
               sp_dict['chain'] = d.should_chain.split(',')
               sp_dict['_chain'] = [sp_expectation('SHALL') for c in d.should_chain.split(',')]

            print(f'======================= SP = {sp_dict["id"]} =====================')
            #print(dumps(sp_dict,indent=4))
                  
            #================ add narrative =======================
            template = env.get_template(md_template[3])   
            sp_text = template.render(sp=sp_dict)
            #print(sp_text)     
            display(HTML(sp_text))
                  
            sp_dict['text'] = {}
            sp_dict['text']['status'] = "generated"
            sp_dict['text']['div'] = sp_text      
                  
            # ================ save files as resource ======================
           #save in ig_source folder
            path = Path.cwd() / ig_source_path / 'resources' / f'searchparameter-{sp_dict["id"]}.json'
            if d.base not in skip_types:    #don't write test types  
                path.write_text(dumps(sp_dict,indent=4))
            sp_list.append(sp_dict)

=========!Patient,address==============
=========!Patient,telecom==============
=========AllergyIntolerance,clinical-status==============
AllergyIntolerance.clinicalStatus
======================= SP = us-core-allergyintolerance-clinical-status =====================


=========AllergyIntolerance,patient==============
======================= SP = us-core-allergyintolerance-patient =====================


=========Condition,category==============
Condition.category
======================= SP = us-core-condition-category =====================


=========Condition,clinical-status==============
Condition.clinicalStatus
======================= SP = us-core-condition-clinical-status =====================


=========Condition,patient==============
======================= SP = us-core-condition-patient =====================


...

=========Condition,code==============
======================= SP = us-core-condition-code =====================


=========Immunization,patient==============
======================= SP = us-core-immunization-patient =====================


=========Immunization,status==============
Immunization.status
======================= SP = us-core-immunization-status =====================


=========Immunization,date==============
======================= SP = us-core-immunization-date =====================


=========!EXAMPLE PATIENT SEARCH,patient==============


### Validate

In [9]:
for i in sp_list:
    print(f'Validating {i["id"]}...........')
    r = validate(i)
    display(HTML(f'<h1>Validation output</h1><h3>Status Code = {r.status_code}</h3> {r.json()["text"]["div"]}'))               

Validating us-core-allergyintolerance-clinical-status...........


Validating us-core-allergyintolerance-patient...........


Validating us-core-condition-category...........


Validating us-core-condition-clinical-status...........


Validating us-core-condition-patient...........


...

Validating us-core-condition-onset-date...........


Validating us-core-condition-code...........


Validating us-core-immunization-patient...........


Validating us-core-immunization-status...........


Validating us-core-immunization-date...........


### Create Quick Start pages using Jinja
 
- spreadsheet for sp and combos

In [10]:
search_type = dict(
    token = '{[system]}|[code]',
    id = '[id]',
    reference = '[reference]',
    string = '[string]',
    uri = '[uri]',
    date = '[date]',
    )



template = env.get_template(md_template[0])

print(r_type)   
for t in r_type:  # preprocess the for jinja templates
    #t = 'AllergyIntolerance'

    sp = [d for d in data if d.base == t]
    sp_combos = [d for d in combo_data if d.base == t]
    #print(sp_combos[0].base)
    mods= {}
    rels = {}
    for s in sp:
            l1=s.shall_modifier.split(',') if s.shall_modifier else []
            l2=s.should_modifier.split(',') if s.should_modifier else []
            l3=s.shall_comparator.split(',') if s.shall_comparator else []
            l4=s.should_comparator.split(',') if s.should_comparator else []
            l5=s.shall_chain.split(',') if s.shall_chain else []
            l6=s.should_chain.split(',') if s.should_chain else []
            mods[s.code] = (l2+l2,l3+l4,l5+l6)
            rels[s.code] = s.rel_url
    
    #print(mods, rels)
    

    shalls = "SHALL" in [i.base_conf for i in sp] + [i.combo_conf for i in sp_combos]  # TODO need to search both singles and combos
    shoulds = "SHOULD" in [i.base_conf for i in sp] + [i.combo_conf for i in sp_combos]
    #pprint(rels)
  
    search_md = template.render(
                    r_type=t,
                    sp=sp,
                    search_type=search_type,
                    combos=sp_combos,
                    shalls=shalls,
                    shoulds=shoulds,
                    mods = mods,
                    rels = rels,
                    )

    
    display(Markdown(search_md))
    # save
    if d.base not in skip_types:
        path = Path.cwd() / ig_source_path / 'pages' / '_includes' / f'us-core-{t.lower()}-search.md'
        path.write_text(search_md)

{'Patient', 'Condition', 'Questionnaire', 'Immunization', 'AllergyIntolerance', 'Encounter'}



#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching a Patient using the **[`_id`](SearchParameter-us-core-patient-_id.html)** search parameter:

    `GET [base]/Patient[id]`

    Example: GET [base]/Patient/1032702

    *Implementation Notes:* Fetches a single Patient (how to search by the [logical id] of the resource)

1. **SHALL** support searching a Patient by an identifier such as a MPI using the **[`identifier`](SearchParameter-us-core-patient-identifier.html)** search parameter:

    `GET [base]/Patient?identifier={[system]}|[code]`

    Example: GET [base]/Patient?identifier=http://hospital.smarthealthit.org\|1032702

    *Implementation Notes:* Fetches a bundle containing any Patient resources matching the identifier (how to search by [token])

1. **SHALL** support searching for a patient by a string match of any part of name using the **[`name`](SearchParameter-us-core-patient-name.html)** search parameter:

    `GET [base]/Patient?name=[string]`

    Example: GET [base]/Patient?name=Shaw

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the name (how to search by [string])

1. **SHOULD** support searching using the combination of the **[`birthdate`](SearchParameter-us-core-patient-birthdate.html)** and **[`name`](SearchParameter-us-core-patient-name.html)** search parameters:

    `GET [base]/Patient?birthdate=[date]&name=[string]`

    Example: GET [base]/Patient?name=Shaw&amp;birthdate=2007-03-20

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified birthdate and name (how to search by [birthdate] and [name])

1. **SHOULD** support searching using the combination of the **[`gender`](SearchParameter-us-core-patient-gender.html)** and **[`name`](SearchParameter-us-core-patient-name.html)** search parameters:

    `GET [base]/Patient?gender=[token]&name=[string]`

    Example: GET [base]/Patient?name=Shaw&amp;gender=female

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified gender and name (how to search by [gender] and [name])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`birthdate`](SearchParameter-us-core-patient-birthdate.html)** and **[`family`](SearchParameter-us-core-patient-family.html)** search parameters:

    `GET [base]/Patient?birthdate=[date]&family=[string]`

    Example: GET [base]/Patient?family=Shaw&amp;birthdate=2007-03-20

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified birthdate and family (how to search by [date] and [string])

1. **SHOULD** support searching using the combination of the **[`family`](SearchParameter-us-core-patient-family.html)** and **[`gender`](SearchParameter-us-core-patient-gender.html)** search parameters:

    `GET [base]/Patient?family=[string]&gender=[token]`

    Example: GET [base]/Patient?family=Shaw&amp;gender=female

    *Implementation Notes:* Fetches a bundle of all Patient resources matching the specified family and gender (how to search by [string] and [token])


{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching for all conditions including problems, health concerns, and encounter diagnosis for a patient using the **[`patient`](SearchParameter-us-core-condition-patient.html)** search parameter:

    `GET [base]/Condition?patient=[reference]`

    Example: GET [base]/Condition?patient=1137192

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient (how to search by [reference])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-condition-patient.html)** and **[`clinical-status`](SearchParameter-us-core-condition-clinical-status.html)** search parameters:

    `GET [base]/Condition?patient=[reference]&clinical-status=[token]`

    Example: GET [base/Condition?patient=[id]&amp;clinical-status=active,recurrance,remission

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient and all “active” statuses (active,relapse,remission). This will not return any “entered in error” resources because of the conditional presence of the clinicalStatus element. (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-condition-patient.html)** and **[`category`](SearchParameter-us-core-condition-category.html)** search parameters:

    `GET [base]/Condition?patient=[reference]&category=[token]`

    Example: GET [base]/Condition?patient=1032702&amp;category=problem,GET [base]/Condition?patient=1032702&amp;category=health-concern,GET [base]/Condition?patient=1032702&amp;category=encounter-diagnosis

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient and category. (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-condition-patient.html)** and **[`code`](SearchParameter-us-core-condition-code.html)** search parameters:

    `GET [base]/Condition?patient=[reference]&code=[token]`

    Example: GET [base]/Condition?patient=[id]&amp;code=[system\|code]

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient and code. (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-condition-patient.html)** and **[`onset-date`](SearchParameter-us-core-condition-onset-date.html)** search parameters:

    `GET [base]/Condition?patient=[reference]&onset-date={gt|lt|ge|le}[date]`

    Example: GET [base]Condition?patient=555580&amp;date=ge2018-01-14

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient and date.  (how to search by [reference] and [date])


{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:




#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.


{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching for all immunizations for a patient using the **[`patient`](SearchParameter-us-core-immunization-patient.html)** search parameter:

    `GET [base]/Immunization?patient=[reference]`

    Example: GET [base]/Immunization?patient=1137192

    *Implementation Notes:* Fetches a bundle of all Immunization resources for the specified patient (how to search by [reference])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-immunization-patient.html)** and **[`date`](SearchParameter-us-core-immunization-date.html)** search parameters:

    `GET [base]/Immunization?patient=[reference]&date={gt|lt|ge|le}[date]`

    Example: GET [base]/Immunization?patient=1137192&amp;date=ge2019-01-14

    *Implementation Notes:* Fetches a bundle of all Immunization resources for the specified patient and date (how to search by [date] and [reference])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-immunization-patient.html)** and **[`status`](SearchParameter-us-core-immunization-status.html)** search parameters:

    `GET [base]/Immunization?patient=[reference]&status=[token]`

    Example: GET [base]/Immunization?patient=1137192&amp;status=completed

    *Implementation Notes:* Fetches a bundle of all Immunization resources for the specified patient and status (how to search by [reference] and [token])


{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support searching for all allergies for a patient using the **[`patient`](SearchParameter-us-core-allergyintolerance-patient.html)** search parameter:

    `GET [base]/AllergyIntolerance?patient=[reference]`

    Example: GET [base]/AllergyIntolerance?patient=1137192

    *Implementation Notes:* Fetches a bundle of all AllergyIntolerance resources for the specified patient (how to search by [reference])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-allergyintolerance-patient.html)** and **[`clinical-status`](SearchParameter-us-core-allergyintolerance-clinical-status.html)** search parameters:

    `GET [base]/AllergyIntolerance?patient=[reference]&clinical-status=[token]`

    Example: GET [base]/AllergyIntolerance?patient=[id]&amp;clinical-status=active

    *Implementation Notes:* Fetches a bundle of all Condition resources for the specified patient and status code.  This will not return any “entered in error” resources because of the conditional presence of the clinicalStatus element. (how to search by [reference] and [token])


{% include link-list.md %}


#### Mandatory Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHALL be supported.  the  modifiers, comparators and chained parameters that are listed as optional SHOULD be supported.:


1. **SHALL** support fetching an Encounter using the **[`_id`](SearchParameter-us-core-encounter-_id.html)** search parameter:

    `GET [base]/Encounter[id]`

    Example: GET [base]/Encounter/12354

    *Implementation Notes:* Fetches a single Encounter (how to search by the [logical id] of the resource)

1. **SHALL** support searching for all encounters for a patient using the **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameter:

    `GET [base]/Encounter?patient=[reference]`

    Example: GET [base]/Encounter?patient=1137192

    *Implementation Notes:* Fetches a bundle of all Encounter resources for the specified patient (how to search by [reference])

1. **SHOULD** support searching using the combination of the **[`date`](SearchParameter-us-core-encounter-date.html)** and **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameters:

    `GET [base]/Encounter?date={gt|lt|ge|le}[date]&patient=[reference]`

    *Implementation Notes:*  (how to search by [date] and [patient])



#### Optional Search Parameters:

The following search parameters, search parameter combinations and search parameter [modifiers], [comparators] and [chained parameters] SHOULD be supported.

1. **SHOULD** support searching using the **[`identifier`](i.rel_url)** search parameter:

    `GET [base]/Encounter?identifier={[system]}|[code]`

    Example: GET [base]/Encounter?identifier=http://hospital.smarthealthit.org\|1032702

    *Implementation Notes:* Fetches a bundle containing any Encounter resources matching the identifier (how to search by [token])

1. **SHOULD** support searching using the combination of the **[`class`](SearchParameter-us-core-encounter-class.html)** and **[`date`](SearchParameter-us-core-encounter-date.html)** search parameters:

    `GET [base]/Encounter?class=[token]&date={gt|lt|ge|le}[date]`

    *Implementation Notes:*  (how to search by [date] and [token])

1. **SHOULD** support searching using the combination of the **[`class`](SearchParameter-us-core-encounter-class.html)** and **[`patient`](SearchParameter-us-core-encounter-patient.html)** search parameters:

    `GET [base]/Encounter?class=[token]&patient=[reference]`

    *Implementation Notes:*  (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-encounter-patient.html)** and **[`status`](SearchParameter-us-core-encounter-status.html)** search parameters:

    `GET [base]/Encounter?patient=[reference]&status=[token]`

    *Implementation Notes:*  (how to search by [reference] and [token])

1. **SHOULD** support searching using the combination of the **[`patient`](SearchParameter-us-core-encounter-patient.html)** and **[`type`](SearchParameter-us-core-encounter-type.html)** search parameters:

    `GET [base]/Encounter?patient=[reference]&type=[token]`

    *Implementation Notes:*  (how to search by [reference] and [token])


{% include link-list.md %}

### Create Markdown Text for SearchParameters Page

- Using Jinja2 Template create markdown file for searchparameters page
- Use spreadsheet as source

In [11]:
for d in data:
    if '!' in d.base:
        print(d.base,True)

# get id for each row in spreadsheet:
r_list={ d.base for d in data if '!' not in d.base and d.base not in skip_types}
d_tup = []
for d in data:
    if d.base in r_list:
        sp_id= [sp['id'] for sp in sp_list if d.base in sp['base'] and sp['code'] in [d.code,'id'] ]
        d_tup.append((sp_id[0] if sp_id else None,d))

template = env.get_template(md_template[1])   

searchparameters_md = template.render(sp_list=d_tup,r_list=r_list)

display(Markdown(searchparameters_md))
# save in pages folder
path = Path.cwd() / ig_source_path / 'pages' / 'searchparameters.md'
path.write_text(searchparameters_md)

!Patient True
!Patient True
!EXAMPLE PATIENT SEARCH True


---
title: Operations and Search Parameters
layout: default
topofpage: true
sectionnumbering: true
---

The following operations and search parameters have been defined for the US Core Implementation Guide.  For more information on the [FHIR RESTful operations], [FHIR RESTful search api] and the standard [Search Param Registry] see the FHIR specification.

### Operations


  {% include list-simple-operationdefinitions.xhtml %}


<br />

### Search Parameters

NOTE: These search parameters **SHALL NOT** be interpreted on their own as requirements for server:
This section lists the search parameters definitions that are used within the capability statement to define the search capabilities expectations for the US Core Server.  This is not a list of search requirements for the client or server.


**Patient**
  - [_id](SearchParameter-us-core-patient-id.html)
  - [birthdate](SearchParameter-us-core-patient-birthdate.html)
  - [family](SearchParameter-us-core-patient-family.html)
  - [gender](SearchParameter-us-core-patient-gender.html)
  - [given](SearchParameter-us-core-patient-given.html)
  - [identifier](SearchParameter-us-core-patient-identifier.html)
  - [name](SearchParameter-us-core-patient-name.html)

**Condition**
  - [category](SearchParameter-us-core-condition-category.html)
  - [clinical-status](SearchParameter-us-core-condition-clinical-status.html)
  - [patient](SearchParameter-us-core-condition-patient.html)
  - [onset-date](SearchParameter-us-core-condition-onset-date.html)
  - [code](SearchParameter-us-core-condition-code.html)

**Immunization**
  - [patient](SearchParameter-us-core-immunization-patient.html)
  - [status](SearchParameter-us-core-immunization-status.html)
  - [date](SearchParameter-us-core-immunization-date.html)

**AllergyIntolerance**
  - [clinical-status](SearchParameter-us-core-allergyintolerance-clinical-status.html)
  - [patient](SearchParameter-us-core-allergyintolerance-patient.html)

**Encounter**
  - [_id](SearchParameter-us-core-encounter-id.html)
  - [class](SearchParameter-us-core-encounter-class.html)
  - [date](SearchParameter-us-core-encounter-date.html)
  - [identifier](SearchParameter-us-core-encounter-identifier.html)
  - [patient](SearchParameter-us-core-encounter-patient.html)
  - [status](SearchParameter-us-core-encounter-status.html)
  - [type](SearchParameter-us-core-encounter-type.html)



Editors note : append these to Patient
  - [us-core-race]
  - [us-core-ethnicity]

{% include link-list.md %}

2494